## Psyco Examples

In [97]:
import os

import matplotlib as plt

import psycopg2
import shapefile
from shapely.geometry import Point, MultiPoint, Polygon
from shapely.wkb import loads

## Import Point Shapefile to Spatial Database

In [ ]:
pg_host = os.environ["PG_SERVER"]
pg_db = os.environ["PG_DATABASE"]
pg_user = os.environ["PG_UID"]
pg_pwd = os.environ["PG_PWD"]

ddl = """
DROP TABLE IF EXISTS public.nyc_museums;

CREATE TABLE public.nyc_museums
(
    id SERIAL NOT NULL 
        PRIMARY KEY, 
    NAME VARCHAR(255) NULL,
    URL VARCHAR(255) NULL,
    location    GEOMETRY(POLYGON, 4326)   NULL
    --location    GEOGRAPHY(POINT, 4326)   NULL
);

CREATE INDEX nyc_museums_points_gix 
ON public.nyc_museums 
USING GIST (location);
"""
point_cloud = []

with psycopg2.connect(host=pg_host, database=pg_db,user=pg_user, password=pg_pwd) as c, \
    shapefile.Reader(nyc_path) as nyc_museums:

    try:
        # Execute DDL
        cursor = c.cursor()
        cursor.execute(ddl)
        c.commit()

        # Execute DML
        cursor = c.cursor()

        dml = """
        INSERT INTO public.nyc_museums (NAME, URL, location)
        VALUES (%s, %s, ST_GeomFromText(%s, 4326))
        """

        # Read both attributes and geometries.
        for srec in nyc_museums.shapeRecords():
            p = Point(srec.shape.points[0][0], srec.shape.points[0][1])
            point_cloud.append(p)
            cursor.execute(dml, (srec.record['NAME'], srec.record['URL'], p.wkt))

        c.commit()
        
    except Exception as e:
        c.rollback()
        raise e

print("Completed.")
mpt = MultiPoint(point_cloud)
mpt

## Read Points from Spatial Database

In [82]:
dml = """
SELECT id
		, name
		, url
		, location 
FROM	public.nyc_museums
LIMIT 10;
"""

with psycopg2.connect(host=pg_host, database=pg_db,user=pg_user, password=pg_pwd) as c:
    cursor = c.cursor()

    cursor.execute(dml)
    
    for rec in cursor.fetchall():
        pt = loads(rec[3], hex=True)
        print(pt)

POINT (-74.01375579573734 40.70381655004537)
POINT (-74.06303178821885 40.61512117086315)
POINT (-73.94729768457961 40.83385383400615)
POINT (-73.97810302107813 40.76162530521804)
POINT (-74.03968483725012 40.69905659568395)
POINT (-73.9736481646561 40.78082656726701)
POINT (-74.00701187947674 40.72352692494523)
POINT (-73.96597045126352 40.76882456292169)
POINT (-73.99963036836223 40.72112778318635)
POINT (-73.96428395731806 40.76983411641748)


## Import Polygons to Spatial Database

In [136]:
pg_host = os.environ["PG_SERVER"]
pg_db = os.environ["PG_DATABASE"]
pg_user = os.environ["PG_UID"]
pg_pwd = os.environ["PG_PWD"]

ddl = """
DROP TABLE IF EXISTS public.us_states;

CREATE TABLE public.us_states
(
    id SERIAL NOT NULL 
        PRIMARY KEY, 
    NAME VARCHAR(100) NULL,
    CODE CHAR(2) NULL,
    ABBREV CHAR(2) NULL,
    AREA BIGINT NULL,
    -- NAD 83
    -- TODO: Can be MultiPolygon. How to handle both?
    location    GEOMETRY(POLYGON, 4269)   NULL
);

CREATE INDEX us_states_poly_gix 
ON public.us_states
USING GIST (location);
"""
shp_path = os.path.join(os.environ["DATA_DIR"], 'us_states', 'tl_2014_us_state.shp')

with psycopg2.connect(host=pg_host, database=pg_db,user=pg_user, password=pg_pwd) as c, \
    shapefile.Reader(shp_path) as shp:

    try:
        # Execute DDL
        cursor = c.cursor()
        cursor.execute(ddl)
        c.commit()

        # Execute DML
        cursor = c.cursor()

        dml = """
        INSERT INTO public.us_states (NAME, CODE, ABBREV, AREA, location)
        VALUES (%s, %s, %s, %s, ST_GeomFromText(%s, 4269))
        """

        # Read both attributes and geometries.
        for srec in shp.shapeRecords()[:5]:
            if srec.record['STUSPS'] not in ('AK', 'HI'):
                poly = Polygon(srec.shape.points)
                cursor.execute(dml, (srec.record['NAME'], srec.record['STATEFP'], 
                                     srec.record['STUSPS'], srec.record['ALAND'], poly.wkt))

        c.commit()
        
    except Exception as e:
        c.rollback()
        raise e

print("Completed.")


Completed.


## Scratch

In [110]:
shp_path = os.path.join(os.environ["DATA_DIR"], 'us_states', 'tl_2014_us_state.shp')

with shapefile.Reader(shp_path) as shp:
    
    # Attribute Definitions
    for meta in shp.fields:
        print(meta)

    # Geometries
    for i, s in enumerate(shp.shapes()[:2]):
        #print(f"Geometry-{i}: {s.bbox}")
        #print(f"Geometry-{i}: {s.points}")
        print(f"Geometryx-{i}: {s.shapeTypeName}")

        poly = Polygon(s.points)
#         print(poly.wkt)

    # Read both attributes and geometries.
    for srec in shp.shapeRecords()[:2]:
        poly = Polygon(srec.shape.points)
        #print(poly.wkt)
        print(f"Geometry-{srec.record.oid}: {srec.shape.shapeTypeName}")
        #print(f"Record-{srec.record.oid}: {srec.record}")
        print(f"Record-{srec.record.oid}: {srec.record['NAME']}, {srec.record['STATEFP']}, {srec.record['STUSPS']}, {srec.record['ALAND']}")
        

('DeletionFlag', 'C', 1, 0)
['REGION', 'C', 2, 0]
['DIVISION', 'C', 2, 0]
['STATEFP', 'C', 2, 0]
['STATENS', 'C', 8, 0]
['GEOID', 'C', 2, 0]
['STUSPS', 'C', 2, 0]
['NAME', 'C', 100, 0]
['LSAD', 'C', 2, 0]
['MTFCC', 'C', 5, 0]
['FUNCSTAT', 'C', 1, 0]
['ALAND', 'N', 14, 0]
['AWATER', 'N', 14, 0]
['INTPTLAT', 'C', 11, 0]
['INTPTLON', 'C', 12, 0]
Geometryx-0: POLYGON
Geometryx-1: POLYGON
Geometry-0: POLYGON
Record-0: West Virginia, 54, WV, 62266581604
Geometry-1: POLYGON
Record-1: Florida, 12, FL, 138903200855
